In [50]:
import MetaTrader5 as mt5 
import pandas as pd
import numpy as np
from statsmodels.tsa.stattools import adfuller 
from datetime import datetime
import requests
import pandas as pd
import numpy as np
import warnings
import json
warnings.filterwarnings("ignore")
mt5.initialize()
# Replace following with your MT5 Account Login
account=51434456 # 
password="9UpBvVzc"
server = 'ICMarkets-Demo'

def get_rates(pair1, timeframe, x):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, timeframe, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1[['time','open', 'high', 'low', 'close']].set_index('time')

In [51]:
import requests
import xml.etree.ElementTree as ET
import pandas as pd

combined_dfs = {}

# Define the base URL for the BIS Stats API
base_url = 'https://stats.bis.org/api/v1'

count_codes = [['AUDUSD.a','AU'], ['USDCAD.a', 'CA'],
               ['USDCHF.a','CH'], ['GBPUSD.a', 'GB'],
               ['USDJPY.a', 'JP'], ['NZDUSD.a', 'NZ'], 
               ['EURUSD.a', 'XM']]

for country in count_codes:

    # Define the endpoint parameters
    flow = 'BIS,WS_EER_D,1.0'  # Example: Version 1.0 of the WS_EER_M domain, maintained by the BIS
    key = f'D.N.N.{country[1]}'
    start_period = '2000'  # Example: Start year 2000
    end_period = '2023'  # Example: End year 2020
    detail = 'full'  # Example: All data and documentation

    # Construct the endpoint URL
    endpoint_url = f'{base_url}/data/{flow}/{key}/all'

    # Define the query parameters
    query_params = {
        'startPeriod': start_period,
        'endPeriod': end_period,
        'detail': detail
    }

    # Make the GET request
    response = requests.get(endpoint_url, params=query_params)

    # Check for a successful response
    if response.status_code == 200:
        # Assign the text of the response to xml_data
        xml_data = response.text

        # Parse the XML data
        root = ET.fromstring(xml_data)

        # Initialize empty lists to store the data
        time_periods = []
        obs_values = []

        # Iterate through the XML and extract the desired information
        for obs in root.findall(".//Obs"):
            time_period = obs.get('TIME_PERIOD')
            obs_value = obs.get('OBS_VALUE')
            time_periods.append(time_period)
            obs_values.append(obs_value)

        # Create a DataFrame
        df = pd.DataFrame({
            'Time_Period': time_periods,
            'OBS_Value': obs_values
        })

        df['OBS_Value'] = df['OBS_Value'].replace('NaN', np.nan)
        # Drop rows with NaN values
        df.dropna(subset=['OBS_Value'], inplace=True)
        df['OBS_Value'] = df['OBS_Value'].astype(float)
        df['Time_Period'] = pd.to_datetime(df['Time_Period'])
        
        df = df.set_index('Time_Period')
        print(f"Getting {country[0]}'s rates")
        rates = get_rates(country[0], mt5.TIMEFRAME_D1, 2500)
        
        combined = pd.concat([df[-len(rates):], rates['close']], join = 'outer', axis = 1)
        
        combined_dfs[country[1]] = combined.dropna()

    else:
        print(f'Failed to retrieve data: {response.status_code}')

Getting AUDUSD.a's rates
Getting USDCAD.a's rates
Getting USDCHF.a's rates
Getting GBPUSD.a's rates
Getting USDJPY.a's rates
Getting NZDUSD.a's rates
Getting EURUSD.a's rates


In [52]:
combined_dfs['AU'].iloc[-1]

OBS_Value    102.65000
close          0.65599
Name: 2023-12-12 00:00:00, dtype: float64

In [53]:
# Compute Expected Returns
weekly_dfs = {}
features = ['OBS_Value', 'close']

for name, dfs in combined_dfs.items():
    df = combined_dfs[name]
    # Assuming 'df' is your DataFrame
    df['date'] = pd.to_datetime(df.index)
    df.set_index('date', inplace=True)

    # Resample to get the last value of each week
    weekly = df.resample('W').last()
    weekly_dfs[name] = weekly

for df in weekly_dfs.values():
    df['EER_ret'] = df['OBS_Value'].pct_change()

In [54]:
for df in weekly_dfs.values():
    print(df['EER_ret'].iloc[-1])

-0.0017504619274529842
0.0004950985246066075
0.0007063393960797804
-0.0017371163867979655
-0.006657963446475068
0.0007866273352998832
0.0029310693349504646


In [45]:
last_update = {}

for name, df in weekly_dfs.items():
    last_update[name] = df['EER_ret'].iloc[-1]

In [46]:
last_update.keys()
symbols = ['AUDUSD.a', 'USDCAD.a', 'USDCHF.a', 'GBPUSD.a', 'USDJPY.a', 'NZDUSD.a', 'EURUSD.a']

last_update = {symbols[i]: value for i, (key, value) in enumerate(last_update.items())}

In [47]:
def send_order(symbol, side, lot, comment):
    
    if side.lower() == 'sell':
        order_type = mt5.ORDER_TYPE_SELL
        price = mt5.symbol_info_tick(symbol).bid
    elif side.lower() == 'buy':
        order_type = mt5.ORDER_TYPE_BUY
        price = mt5.symbol_info_tick(symbol).ask
    
    request = {
        "action": mt5.TRADE_ACTION_DEAL,
        "symbol": symbol,
        "volume": lot,
        "type": order_type,
        "price": price,
        "deviation": 5,
        "magic": 234000,
        "comment": comment,
        "type_time": mt5.ORDER_TIME_GTC,
        "type_filling": mt5.ORDER_FILLING_IOC,
    }
    result = mt5.order_send(request)
    result

In [48]:
last_update

{'AUDUSD.a': -0.0017504619274529842,
 'USDCAD.a': 0.0004950985246066075,
 'USDCHF.a': 0.0007063393960797804,
 'GBPUSD.a': -0.0017371163867979655,
 'USDJPY.a': -0.006657963446475068,
 'NZDUSD.a': 0.0007866273352998832,
 'EURUSD.a': 0.0029310693349504646}

In [49]:
for symbol, value in last_update.items():
    if symbol[:3] == 'USD':
        print(f"Looping through {symbol}")
        if value > 0:
            send_order(symbol, 'sell', 1.00, 'EER')
            print(f"Selling {symbol}")
        else:
            send_order(symbol, 'buy', 1.00, 'EER')
    else:
        if value > 0:
            send_order(symbol, 'buy', 1.00, 'EER')
        else:
            send_order(symbol, 'sell', 1.00, 'EER')

Looping through USDCAD.a
Selling USDCAD.a
Looping through USDCHF.a
Selling USDCHF.a
Looping through USDJPY.a


In [21]:
def close_position(position):

    tick = mt5.symbol_info_tick(position.symbol)

    request = {
        "action" : mt5.TRADE_ACTION_DEAL,
        "position": position.ticket,
        "symbol": position.symbol,
        "volume": position.volume,
        "type": mt5.ORDER_TYPE_BUY if position.type == 1 else mt5.ORDER_TYPE_SELL,
        "price": tick.ask if position.type == 1 else tick.bid,
        "deviation": 20,
        "magic": 100,
        "comment": 'Regres Close',
        'type_time': mt5.ORDER_TIME_GTC,
        'type_filling':mt5.ORDER_FILLING_IOC,

        }
    result = mt5.order_send(request)
    
def close_all():
    close_positions = []
    open_positions = mt5.positions_get()
    open_positions
    for i in open_positions:
        close_positions.append(i)
        
    for pos in close_positions:
        close_position(pos)

In [22]:
close_all()

In [38]:
# Work on automating it 

## Research

Analyzing the relationship of EER data and close prices. 

In [55]:
for df in weekly_dfs.values():
    df['ret'] = df['close'].pct_change()
    df['ret_diff'] = df['EER_ret'] - df['ret']
    df = df.dropna()

In [56]:
for name, df in weekly_dfs.items():
    print(f"EER Correlation with {name} is: {df['EER_ret'].corr(df['ret'])}")

EER Correlation with AU is: 0.7901864448535373
EER Correlation with CA is: -0.8028662382263346
EER Correlation with CH is: -0.6972064531265871
EER Correlation with GB is: 0.6872038876573434
EER Correlation with JP is: -0.8162789646117152
EER Correlation with NZ is: 0.7759010905793634
EER Correlation with XM is: 0.7546198496480194


In [64]:
desc = pd.DataFrame()
for name in weekly_dfs.keys():
    desc[f'{name} Ret-Diff Stats'] = weekly_dfs[name]['ret_diff'].describe()
desc

,AU Ret-Diff Stats,CA Ret-Diff Stats,CH Ret-Diff Stats,GB Ret-Diff Stats,JP Ret-Diff Stats,NZ Ret-Diff Stats,XM Ret-Diff Stats
count,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000,500.000000
mean,0.000331,-0.000780,0.000455,0.000355,-0.001187,0.000385,0.000303
std,0.009016,0.018449,0.020908,0.009577,0.022378,0.009825,0.007543
min,-0.053037,-0.064334,-0.104875,-0.050289,-0.100707,-0.055615,-0.033891
25%,-0.005043,-0.011253,-0.009497,-0.005518,-0.013587,-0.004930,-0.004470
50%,0.000351,-0.000383,-0.000585,0.000610,-0.002483,0.000214,0.000593
75%,0.005571,0.011458,0.009036,0.005837,0.010900,0.005794,0.004755
max,0.033373,0.059054,0.284139,0.036804,0.077664,0.039223,0.029442


## Risk Management Framework

Aim is to take into account associated statistics between the two and minimise volatility whilst maximising return. Appropriate measures can include:
- MVO or BL Portfolio
- Correlation analysis and appropriate hedging associated 